# Practica 2: Teleportación Cuántica y Codificación Superdensa

Guillermo García Patiño Lenza

### Ejercicio 3:

#### Teleportación Cuántica:

In [2]:
import numpy as np
from qiskit.visualization import plot_histogram
from qiskit import (
    QuantumCircuit,
    execute,
    Aer)
from qiskit import IBMQ
from qiskit import IBMQ
from qiskit import assemble, transpile
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import QasmSimulator

In [6]:
def creaCircuitoTelep():
    circuit = QuantumCircuit(3,3)
    
    #entrelazar q1 y q2
    circuit.h(1)
    circuit.cx(1,2)
    
    #codificar el estado de psi 
    circuit.cx(0,1)
    circuit.h(0)
    
    #decodificar psi
    circuit.cx(1,2)
    circuit.cz(0,2)
    
    #medir
    circuit.measure([0,1,2],[0,1,2])
    
    return circuit

In [7]:
c = creaCircuitoTelep()
c.draw()

┌───┐      ┌─┐   
q_0: ────────────■──┤ H ├─■────┤M├───
     ┌───┐     ┌─┴─┐└───┘ │ ┌─┐└╥┘   
q_1: ┤ H ├──■──┤ X ├──■───┼─┤M├─╫────
     └───┘┌─┴─┐└───┘┌─┴─┐ │ └╥┘ ║ ┌─┐
q_2: ─────┤ X ├─────┤ X ├─■──╫──╫─┤M├
          └───┘     └───┘    ║  ║ └╥┘
c: 3/════════════════════════╩══╩══╩═
                             1  0  2

In [8]:
def creaCircuitoCodif(b1,b0):
    circuit = QuantumCircuit(2,2)
    
    #entrelazar q0 y q1
    circuit.h(0)
    circuit.cx(0,1)
    
    #codificar bits clasicos en q0
    if(b1 == 1 and b0 == 1):
        circuit.x(0)
        circuit.z(0)
    elif(b1 == 1 and b0 == 0):
        circuit.z(0)
    elif(b1 == 0 and b1 == 1):
        circuit.x(0)
        
    #decodificar bits clasicos
    
    circuit.cx(0,1)
    circuit.h(0)
    
    #medir
    circuit.measure([0,1],[0,1])
    
    return circuit

In [9]:
c2 = creaCircuitoCodif(1,1)
c2.draw()

┌───┐     ┌───┐┌───┐     ┌───┐┌─┐
q_0: ┤ H ├──■──┤ X ├┤ Z ├──■──┤ H ├┤M├
     └───┘┌─┴─┐└───┘└───┘┌─┴─┐└┬─┬┘└╥┘
q_1: ─────┤ X ├──────────┤ X ├─┤M├──╫─
          └───┘          └───┘ └╥┘  ║ 
c: 2/═══════════════════════════╩═══╩═
                                1   0